# Visualization of PASEOS

First load the necessary packages

In [ ]:
%load_ext autoreload
%autoreload 2

# If you haven't installed PASEOS
import sys
sys.path.append("..")
sys.path.append("../..")

import numpy as np

# We use pykep for orbit determination
import pykep as pk

# Code for creating constellations
from utils.get_constellation import get_constellation

import paseos
from paseos.actors.ground_station_actor import GroundstationActor
from paseos.actors.spacecraft_actor import SpacecraftActor
from paseos.actors.actor_builder import ActorBuilder
paseos.set_log_level("INFO")

Create a PASEOS object. Add some actors

In [ ]:
# Define a constellation
altitude = 786 * 1000  # altitude above the Earth's ground [m]
inclination = 98.62  # inclination of the orbit
nPlanes = 1 # the number of orbital planes (see linked wiki article)
nSats = 2 # the number of satellites per orbital plane
t0 = pk.epoch_from_string("2022-Dec-17 14:42:42")  # starting date of our simulation

# Compute orbits of LEO satellites
planet_list,sats_pos_and_v,_ = get_constellation(altitude,inclination,nSats,nPlanes,t0)

paseos_instances = [] # this will store paseos instances
earth = pk.planet.jpl_lp("earth") # define our central body

for idx,sat_pos_v in enumerate(sats_pos_and_v):
    pos,v = sat_pos_v
    sat_actor = ActorBuilder.get_actor_scaffold(name=["Sentinel_2A","Sentinel_2B"][idx],
                                                actor_type=SpacecraftActor,
                                                epoch=t0)
    ActorBuilder.set_orbit(actor=sat_actor,position=pos,velocity=v,epoch=t0,central_body=earth)
    ActorBuilder.add_comm_device(actor=sat_actor,device_name="Link1",bandwidth_in_kbps=1000)
    ActorBuilder.set_power_devices(actor=sat_actor,battery_level_in_Ws=10000+np.random.rand()*90000,
                                   max_battery_level_in_Ws=100000,charging_rate_in_W=50)
    ActorBuilder.set_thermal_model(
        actor=sat_actor,
        actor_mass=6.0,
        actor_initial_temperature_in_K=283.15,
        actor_sun_absorptance=0.9,
        actor_infrared_absorptance=0.5,
        actor_sun_facing_area=0.012,
        actor_central_body_facing_area=0.01,
        actor_emissive_area=0.125,
        actor_thermal_capacity=6000,
    )
    
    instance = paseos.init_sim(local_actor=sat_actor)
    paseos_instances.append(instance)
    if idx > 0:
        paseos_instance.add_known_actor(sat_actor)
    else:
        paseos_instance = instance
        
# Ground stations
stations = [
    ["Maspalomas", 27.7629, -15.6338, 205.1],
    ["Matera", 40.6486, 16.7046, 536.9],
    ["Svalbard", 78.9067, 11.8883, 474.0],
]
groundstation_actors = []
for station in stations:
    gs_actor = ActorBuilder.get_actor_scaffold(
        name=station[0], actor_type=GroundstationActor, epoch=t0
    )
    ActorBuilder.set_ground_station_location(
        gs_actor,
        latitude=station[1],
        longitude=station[2],
        elevation=station[3],
        minimum_altitude_angle=5,
    )
    paseos_instance.add_known_actor(gs_actor)
    groundstation_actors.append(gs_actor)

In [ ]:
# Plot current status of PASEOS and get a plotter
%matplotlib inline
plotter = paseos.plot(paseos_instance, paseos.PlotType.SpacePlot)

In [ ]:
# Run some operations and inbetween update PASEOS
for i in range(100):
    for idx,instance in enumerate(paseos_instances):
        instance.advance_time(100,np.random.randint(50))
        if idx > 0:
            paseos_instance.add_known_actor(instance.local_actor)
        else:
            paseos_instance.empty_known_actors()
            for station in groundstation_actors:
                paseos_instance.add_known_actor(station)
    plotter.update(paseos_instance)

In [ ]:
%%time
import matplotlib.animation as animation

def anim_func(_,dt,paseos_instances):
    # Run some operations and inbetween update PASEOS
    for idx,instance in enumerate(paseos_instances):
        instance.advance_time(dt,np.random.randint(50))
        if idx > 0:
            paseos_instance.add_known_actor(instance.local_actor)
        else:
            paseos_instance.empty_known_actors()
            for station in groundstation_actors:
                paseos_instance.add_known_actor(station)
    plotter.update(paseos_instance, creating_animation=True)
    return plotter.ax_3d.get_children() + plotter.ax_los.get_children()

anim = animation.FuncAnimation(
            plotter.fig,
            anim_func,
            frames=60,
            fargs=(
                10,paseos_instances
            ),
            interval=20,
            blit=True,
        )  # blit means to only redraw parts that changed
anim.save("testrun.mp4", writer="ffmpeg", fps=20)